---

### <center> <b> <font color = 'green'> Aplicação de Pooling </font> <center>  

### <center> <b> <font color = 'white'>🌳Convolutional Neural Networks - CNNs🌳 </font> <center> 

---

In [1]:
#  import these dependencies
import torch
import numpy as np


ModuleNotFoundError: No module named 'torch'

In [ ]:
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def max_pool(image_path, kernel_size=2, visualize=False, title=''):
      """
      This function replicates the maxpooling
      process
      """
      
      #  assessing image parameter
      if type(image_path) is np.ndarray and len(image_path.shape)==2:
        image = image_path
      else:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
      
      #  creating an empty list to store convolutions
      pooled = np.zeros((image.shape[0]//kernel_size, 
                        image.shape[1]//kernel_size))
                        
      #  instantiating counter
      k=-1
      #  maxpooling
      for i in tqdm(range(0, image.shape[0], kernel_size)):
        k+=1
        l=-1
        if k==pooled.shape[0]:
          break
        for j in range(0, image.shape[1], kernel_size):
          l+=1
          if l==pooled.shape[1]:
            break
          try:
            pooled[k,l] = (image[i:(i+kernel_size), 
                                j:(j+kernel_size)]).max()
          except ValueError:
            pass
            
      if visualize:
        #  displaying results
        figure, axes = plt.subplots(1,2, dpi=120)
        plt.suptitle(title)
        axes[0].imshow(image, cmap='gray')
        axes[0].set_title('Imagem Referência')
        axes[1].imshow(pooled, cmap='gray')
        axes[1].set_title('Max pooling')
      return pooled


In [ ]:
max_pool('image.png', kernel_size=2, visualize=True)
# diminuiu de tamanho, mas manteve todos os seus detalhes intactos
# É quase como se o processo tivesse extraído os pixels mais salientes e produzido uma representação resumida com metade do tamanho da imagem de referência (metade porque um kernel (2, 2) foi usado).
# Imagem original: 527 X 1280

In [ ]:
# Max Pooling
def max_pool(image_path, kernel_size=2, iterations=3, save_path='pooling_results.png'):
    """
    Aplica múltiplas iterações de max pooling na imagem, gera resultados lado a lado,
    exibe as escalas nos eixos e adiciona o shape da imagem original e de cada resultado.
    """

    # Carregar a imagem
    if isinstance(image_path, np.ndarray) and len(image_path.shape) == 2:
        image = image_path
    else:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    original_shape = image.shape
    # Lista para armazenar as imagens resultantes
    pooled_images = [image]

    # Função de max pooling que trata limites
    def max_pooling(input_image, kernel_size):
        h, w = input_image.shape[:2]
        new_h = h // kernel_size
        new_w = w // kernel_size
        pooled = np.zeros((new_h, new_w, *input_image.shape[2:]), dtype=input_image.dtype)
        for i in range(new_h):
            for j in range(new_w):
                start_i = i * kernel_size
                start_j = j * kernel_size
                end_i = start_i + kernel_size
                end_j = start_j + kernel_size
                window = input_image[start_i:end_i, start_j:end_j]
                pooled[i, j] = window.max(axis=(0, 1))
        return pooled

    # Iterar para aplicar pooling várias vezes
    current_image = image
    for _ in range(iterations):
        pooled_image = max_pooling(current_image, kernel_size)
        if len(current_image.shape) == 3:
            pooled_image = np.clip(pooled_image, 0, 255).astype(np.uint8)
        else:
            pooled_image = np.clip(pooled_image, 0, 255).astype(np.uint8)
        pooled_images.append(pooled_image)
        current_image = pooled_image

    # Criar visualização lado a lado, adicionando informações de shape
    fig, axes = plt.subplots(1, len(pooled_images), figsize=(15, 5))
    for idx, img in enumerate(pooled_images):
        axes[idx].imshow(img)
        axes[idx].set_xlabel('Pixels')
        axes[idx].set_ylabel('Pixels')
        # Adicionar o shape da imagem original e da atual
        shape_info = f'Shape: {original_shape}' if idx == 0 else f'Shape: {img.shape}'
        axes[idx].set_title(f'{"Original" if idx==0 else f"Pool {idx}"}\n{shape_info}')
        axes[idx].axis('on')  # Mostrar os eixos

    plt.suptitle('Max Pooling Iterations')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    return pooled_images


In [ ]:
max_pool('image.png', kernel_size=2, iterations=3, save_path='max_poll.png')

In [ ]:
# Average Pooling

def average_pool(image_path, kernel_size=2, iterations=3, save_path='average_pooling_results.png'):
    """
    Aplica múltiplas iterações de average pooling na imagem, gera resultados lado a lado,
    exibe as escalas nos eixos, e inclui o shape da imagem original e de cada resultado.
    """

    # Carregar a imagem
    if isinstance(image_path, np.ndarray) and len(image_path.shape) == 2:
        image = image_path
    else:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    original_shape = image.shape
    # Lista para armazenar as imagens resultantes
    pooled_images = [image]

    # Função de average pooling que trata limites
    def average_pooling(input_image, kernel_size):
        h, w = input_image.shape[:2]
        new_h = h // kernel_size
        new_w = w // kernel_size
        pooled = np.zeros((new_h, new_w, *input_image.shape[2:]), dtype=input_image.dtype)
        for i in range(new_h):
            for j in range(new_w):
                start_i = i * kernel_size
                start_j = j * kernel_size
                end_i = start_i + kernel_size
                end_j = start_j + kernel_size
                window = input_image[start_i:end_i, start_j:end_j]
                pooled[i, j] = window.mean(axis=(0, 1))
        return pooled

    # Iterar para aplicar average pooling várias vezes
    current_image = image
    for _ in range(iterations):
        pooled_image = average_pooling(current_image, kernel_size)
        if len(current_image.shape) == 3:
            pooled_image = np.clip(pooled_image, 0, 255).astype(np.uint8)
        else:
            pooled_image = np.clip(pooled_image, 0, 255).astype(np.uint8)
        pooled_images.append(pooled_image)
        current_image = pooled_image

    # Criar visualização lado a lado, adicionando informações de shape
    fig, axes = plt.subplots(1, len(pooled_images), figsize=(15, 5))
    for idx, img in enumerate(pooled_images):
        axes[idx].imshow(img)
        axes[idx].set_xlabel('Pixels')
        axes[idx].set_ylabel('Pixels')
        shape_info = f'Original Shape: {original_shape}' if idx == 0 else f'Shape: {img.shape}'
        axes[idx].set_title(f'{"Original" if idx == 0 else f"Pool {idx}"}\n{shape_info}')
        axes[idx].axis('on')  # Mostrar os eixos

    plt.suptitle('Average Pooling Iterations')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    return pooled_images


In [ ]:
average_pool('image.png', kernel_size=2, iterations=3, save_path='aver_pooling.png')